In [0]:
dbutils.library.restartPython()

# Import de Funções e Libs

As bibliotecas e funções utilizadas ao longo do código devem ser adicionadas aqui

In [0]:
import numpy as np
import pandas as pd
import requests
import json
import sherlock
import regras_politica as rp

### Testes Modulares - Referente ao tópico 3 - Tabelas tópico 3.5

### Geração de Massa

In [0]:
#Gerando a massa modelo passando o caminho do JSON como nome da variável
base_entrada = pd.DataFrame({'payloadHomol.intermediarias.faixaRiscoInternoAgrupada':["R10"],
                             'payloadHomol.intermediarias.segmentacaoPolitica':["P1 - Funcionarios"],
                             'payloadHomol.intermediarias.faixaRendaBruta':["d. >= 15k"],
                             'solicitante.flagFuncionario':[0],
                             'payloadHomol.intermediarias.cpf5e6Digito':[99],
                             'solicitante.flagCCRot':[1],
                             'solicitante.mediaSaldoTotal':[40000]
                            })

In [0]:
#Criação das massas com casos forçados
massa_TabelaJaCliente1 = sherlock.Gera_Massa(base_entrada)
massa_TabelaJaCliente1.gera_cenario('payloadHomol.intermediarias.faixaRiscoInternoAgrupada',["R1","R2","R3",
                                                                                            "R4","R5","R6",
                                                                                            "R7","R8","R9",
                                                                                            "R10"])
massa_TabelaJaCliente1.gera_cenario('payloadHomol.intermediarias.segmentacaoPolitica',["P1 - Funcionarios",
                                                                                      "P2 - FOPAG",
                                                                                      "P3 - Investidor Alta Renda",
                                                                                      "P4.1 - Alta Renda Principalidade",
                                                                                      "P4.2 - Alta Renda MAT",
                                                                                      "P4.3 - Alta Renda Inativo",
                                                                                      "P5.4 - Varejo+ Principalidade Renda < 4k",
                                                                                      "P5.1 - Varejo+ Principalidade Renda >= 4k",
                                                                                      "P5.2 - Varejo+ MAT",
                                                                                      "P5.3 - Varejo+ Inativos",
                                                                                      "P6.1 - Varejo MAT",
                                                                                      "P6.2 - Varejo Inativos",
                                                                                      "P6.2 - Varejo Inativos"])
massa_TabelaJaCliente1.gera_cenario('payloadHomol.intermediarias.faixaRendaBruta',['a. Sem renda',
                                                                                  'a. < 4k',
                                                                                  'b. 4k - 10k',
                                                                                  'c. 10k - 15k',
                                                                                  'd. >= 15k',
                                                                                  'z. Verificar'])
massa_TabelaJaCliente1.gera_cenario('solicitante.flagFuncionario',[0,1])
massa_TabelaJaCliente1.gera_cenario('payloadHomol.intermediarias.cpf5e6Digito',[0,43,44,45,46,88,89,90,91,99])
massa_TabelaJaCliente1 = massa_TabelaJaCliente1.get_massa_final()

# Teste de aplicação da coluna de rotativo e mean total balance para clientes investidor alta renda
massa_TabelaJaCliente2 = sherlock.Gera_Massa(base_entrada)
massa_TabelaJaCliente2.gera_cenario('payloadHomol.intermediarias.faixaRiscoInternoAgrupada',["R1","R2","R3",
                                                                                            "R4","R5","R6",
                                                                                            "R7","R8","R9",
                                                                                            "R10"])
massa_TabelaJaCliente2.gera_cenario('payloadHomol.intermediarias.segmentacaoPolitica',["P1 - Funcionarios",
                                                                                       "P3 - Investidor Alta Renda",
                                                                                       "P6.2 - Varejo Inativos"])
massa_TabelaJaCliente2.gera_cenario('payloadHomol.intermediarias.faixaRendaBruta',['a. Sem renda',
                                                                                   'a. < 4k',
                                                                                   'd. >= 15k'])
massa_TabelaJaCliente2.gera_cenario('solicitante.flagFuncionario',[0,1])
massa_TabelaJaCliente2.gera_cenario('payloadHomol.intermediarias.cpf5e6Digito',[0,99])
massa_TabelaJaCliente2.gera_cenario('solicitante.flagCCRot',[0,1])
massa_TabelaJaCliente2.gera_cenario('solicitante.mediaSaldoTotal',[39999,40000,40001])
massa_TabelaJaCliente2 = massa_TabelaJaCliente2.get_massa_final()

#Empilha todos casos com 1 caso padrão.
massa_TabelaJaCliente = sherlock.Gera_Massa(base_entrada)
massa_TabelaJaCliente.empilha_massas([massa_TabelaJaCliente1,
                                      massa_TabelaJaCliente2
])

massa_TabelaJaCliente.get_massa_final()

massa_TabelaJaCliente.set_bom_arqv()
massa_TabelaJaCliente.converte_massa()

### Execução Modular e Geração de CSV

In [0]:
listaResultado = rp.execucaoModular(massa_TabelaJaCliente.get_massa_json(),rp.subGruposAntigo)

##### Valida se todas condições estão sendo abordadas

In [0]:
# buscando um perfil de cliente para análise
for payload in listaResultado:
    if (payload['payloadHomol']['intermediarias']['faixaRiscoInternoAgrupada']  == "R1"
        and
        payload['payloadHomol']['intermediarias']['segmentacaoPolitica'] == "P3 - Investidor Alta Renda"
        and 
        payload['payloadHomol']['intermediarias']['faixaRendaBruta'] == "a. < 4k"
        and
        payload['solicitante']['flagFuncionario'] == 0
        and 
        payload['solicitante']['flagCCRot'] == 0
        ):
        print(json.dumps(payload,indent = 4))
        break

In [0]:
# mudando o max de display de linhas dos dataframes desse notebook
pd.set_option('display.max_rows', 300) 
import pandas as pd

# fazendo freq para analisar se todos os cenários da tabela são abordados na massa
results = []
for caso in listaResultado:
    results.append(caso['payloadHomol']['intermediarias']['cenario'])
    
freqzinha = pd.DataFrame(results, columns=['valor'])
freqzinha['Freq'] = freqzinha['valor'].map(freqzinha['valor'].value_counts())
freqzinha = freqzinha.drop_duplicates().reset_index(drop=True)
freqzinha.sort_values(by='valor', ascending=True)

In [0]:
# Renomeação das colunas para o cabeçalho do RMA
massa_TabelaJaCliente.rename_cabecalho()
massa_TabelaJaCliente.rename_variavel("solicitante.flagCCRot","payloadParam.solicitante.flagCCRot")
massa_TabelaJaCliente.rename_variavel("solicitante.mediaSaldoTotal","payloadParam.solicitante.mediaSaldoTotal")

In [0]:
massa_TabelaJaCliente.get_massa_dm()

In [0]:
massa_TabelaJaCliente.atribui_expected(listaResultado,"payloadHomol.intermediarias.segmentacao","segmentacaoSubgrupo")
massa_TabelaJaCliente.atribui_expected(listaResultado,"payloadHomol.intermediarias.alavancagem","alavancagem")
massa_TabelaJaCliente.atribui_expected(listaResultado,"payloadHomol.intermediarias.limiteTeto","limiteTeto")
massa_TabelaJaCliente.atribui_expected(listaResultado,"payloadHomol.intermediarias.limitePiso","limitePiso")
massa_TabelaJaCliente.atribui_expected(listaResultado,"payloadHomol.intermediarias.limiteFixo","limiteFixo")
massa_TabelaJaCliente.atribui_expected(listaResultado,"payloadHomol.intermediarias.aprovadoBAUAux","aprovadoBAUAux")
massa_TabelaJaCliente.atribui_expected(listaResultado,"payloadHomol.intermediarias.aprovadoSLAux","aprovadoSmallLimitsAux")

massa_TabelaJaCliente.get_massa_dm()

### Conversão da massa para CSV

In [0]:
massa_TabelaJaCliente.get_massa_dm().to_csv(r'/Workspace/Users/kaue.ongaro@picpaybank.com/2 - Homologação/Concessao de Cartoes - Online/massas/massaDefinicaoTabelasSubGruposAntigos.csv',index=False)

In [0]:
dataframeGrande = massa_TabelaJaCliente.get_massa_dm()
dataframeGrande

In [0]:
data_id = 6002
print(json.dumps(listaResultado[data_id-1],indent=4))